# LangChain: Agents

## 목차:

* LangChain에 내장된 tool 사용하기 : DuckDuckGo search and Wikipedia
* 나만의 tool 정의하기

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

## LangChain 내장 tool

In [ ]:
#!pip install -U wikipedia

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0)

LangChain이 원래 가지고 있는 도구들을 `load_tools` 함수로 불러옵니다.  
이때 verbose 옵션은 명령 수행 과정을 시각적으로 확인할 수 있게 해주는 옵션입니다.

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

이제 정의된 agent에 입력만 제공하면 query에 알맞는 tool을 선택해서 답변을 제공합니다.

In [ ]:
agent("What is the 25% of 300?")

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 

## Python Agent

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

agent가 받을 입력에 f-string 방식으로 변수를 전달할 수 있습니다.  
`customer_list`에 다양한 정보가 입력될 수 있음을 확인하세요.

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

#### chain들의 상세한 output을 직접 확인하고 싶으면 `debug` 옵션을 True로 설정하면 됩니다.

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

## 나만의 도구 만들기

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

기존에 파이썬에서 사용할 수 있는 datetime 모듈을 불러와 `time` 함수를 정의합니다.  
이때 langchain의 <span style='color:red'>tool 데코레이터</span>를 사용합니다.

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**주목**: 

agent가 때론 잘못된 결론을 내릴 수도 있습니다.  
그런 경우에는 다시 시도해보세요!

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")